<a href="https://colab.research.google.com/github/sugarlin1732/RNN_Drug_property-/blob/main/02_crawl_sml_property.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from os import sep
import requests
from bs4 import BeautifulSoup
import re
import time
import numpy as np
import pandas as pd

In [ ]:
# connect to google drive
from google.colab import  drive
drive.mount('/content/drive')
# change work dir
path = '/content/drive/My Drive/Drug_RNN'
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#######################################################################
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
# load drug id
with open('./drugbank_id.txt', 'r') as file1:
    id_list = [i.replace('\n', '') for i in file1]

drug_dic = {}
counter = 1

prop_name = ['Absorption', 'Distribution', 'ProteinBinding', 'Metabolism', 'RouteOfElimination', 'Halflife', 'Clearance', 'Toxicity', 'AffectedOrganism', 'FoodInteraction', 'Unii', 'CASnumber', 'InChIKey', 'InChI', 'IUPAC', 'State']
prop_id = ['absorption', 'volume-of-distribution', 'protein-binding', 'metabolism', 'route-of-elimination', 'half-life', 'clearance', 'toxicity', 'affected-organisms', 'food-interactions', 'unii', 'cas-number', 'inchi-key', 'inchi', 'iupac-name', 'state']

# crawl drug property data from DrugBank
if not os.path.exists('./data'):
    os.makedirs('./data')
counter = 1
for drug in id_list[2000:]:
    print(counter, '\r', end='')
    # print(counter, drug, '\r', end='')
    url = 'https://go.drugbank.com/drugs/' + drug
    r1 = requests.get(url, headers=headers).text
    soup = BeautifulSoup(r1, 'lxml')

    prop_list = []

    with open('./data/'+drug+'.txt', 'w', encoding='utf-8') as o1:
        print('DrugBankID', soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4 pl-xl-2', id='accession-number').find_next_siblings()[0].string, sep='\t', file=o1)
        
        try:
            weight_tmp = soup.find('dt', class_='col-xl-4 col-md-3 col-sm-4 pl-xl-2', id='weight').find_next_siblings()[0].text.split(' Monoisotopic: ')
            print('AverageWeight', weight_tmp[0].lstrip('Average: '), sep='\t', file=o1)
            print('MonoisotopicWeight', weight_tmp[-1].rstrip(' '), sep='\t', file=o1)
        except:
            pass

        for i in range(len(prop_name)):
            try:
                print(prop_name[i], soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id = prop_id[i]).find_next_siblings()[0].text.rstrip('\n').lstrip('\n').replace('\n', '; '), sep='\t', file=o1)
            except:
                pass
                
        try:
            result = soup.find('table', id='experimental-properties')
            expp_tmp = []
            for i in result.find_all('tr'):
                for j in i.find_all('td'):
                    expp_tmp.append(j.text)
            for i in range(len(expp_tmp)//3):
                print('ExpProp_'+expp_tmp[i*3], expp_tmp[i*3+1], sep='\t', file=o1)
        except:
            pass

        try:
            result = soup.find('table', id='drug-moldb-properties')
            prep_tmp = []
            for i in result.find_all('tr'):
                for j in i.find_all('td'):
                    prep_tmp.append(j.text)
            for i in range(len(prep_tmp)//3):
                print('PredProp_'+prep_tmp[i*3]+'_'+prep_tmp[i*3+2], prep_tmp[i*3+1], sep='\t', file=o1)
        except:
            pass

        try:
            result = soup.find('table', id='drug-predicted-admet')
            preadmet_tmp = []
            for i in result.find_all('tr'):
                for j in i.find_all('td'):
                    preadmet_tmp.append(j.text)
            for i in range(len(preadmet_tmp)//3):
                print('PredADMET_'+preadmet_tmp[i*3], preadmet_tmp[i*3+1], preadmet_tmp[i*3+2], sep='\t', file=o1)
        except:
            pass

    counter += 1
    time.sleep(20)
print('Complete!')

Complete!


In [ ]:
# DrugBankID              soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4 pl-xl-2', id='accession-number').find_next_siblings()[0].string

# result = soup.find('dt', class_='col-xl-4 col-md-3 col-sm-4 pl-xl-2', id='weight').find_next_siblings()[0].text.split(' Monoisotopic: ')
# AverageWeight           result[0].lstrip('Average: ')
# MonoisotopicWeight      result[-1].rstrip(' ')

# Absorption              result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='absorption').find_next_siblings()[0].text
# Distribution            result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='volume-of-distribution').find_next_siblings()[0].text
# ProteinBinding          result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='protein-binding').find_next_siblings()[0].text
# Metabolism              result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='metabolism').find_next_siblings()[0].text
# RouteOfElimination      result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='route-of-elimination').find_next_siblings()[0].text
# Halflife                result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='half-life').find_next_siblings()[0].text
# Clearance               result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='clearance').find_next_siblings()[0].text
# Toxicity                result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='toxicity').find_next_siblings()[0].text
# AffectedOrganism        result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='affected-organisms').find_next_siblings()[0].text
# FoodInteraction         result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='food-interactions').find_next_siblings()[0].text
# Unii                    result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='unii').find_next_siblings()[0].text
# CASnumber               result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='cas-number').find_next_siblings()[0].text
# InChIKey                result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='inchi-key').find_next_siblings()[0].text
# InChI                   result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='inchi').find_next_siblings()[0].text
# IUPAC                   result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='iupac-name').find_next_siblings()[0].text
# State                   result = soup.find('dt', class_='col-xl-2 col-md-3 col-sm-4', id='state').find_next_siblings()[0].text

# ExpProp                 result = soup.find('table', id='experimental-properties')
#                             for i in result.find_all('tr'):
#                                 for j in i.find_all('td'):
#                                     print(j.text)
# PredProp_               result = soup.find('table', id='drug-moldb-properties')
#                             for i in result.find_all('tr'):
#                                 for j in i.find_all('td'):
#                                     print(j.text)
# PredADMET_               result = soup.find('table', id='drug-predicted-admet')
#                             for i in result.find_all('tr'):
#                                 for j in i.find_all('td'):
#                                     print(j.text)

